In [ ]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np    

In [ ]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
!unzip /content/drive/MyDrive/face-mask-dataset.zip -d /content/ 

Archive:  /content/drive/MyDrive/face-mask-dataset.zip
  inflating: /content/Dataset/test.zip  
  inflating: /content/Dataset/train.zip  


In [ ]:
!unzip /content/Dataset/test.zip -d /content/ 
!unzip /content/Dataset/train.zip -d /content/ 

In [ ]:
TRAINING_DIR = "/content/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [ ]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

<ipython-input-9-6272b23e5a0b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
132/132 [==============================] - ETA: 0s - loss: 0.4768 - acc: 0.7795

132/132 [==============================] - 153s 1s/step - loss: 0.4768 - acc: 0.7795 - val_loss: 0.1335 - val_acc: 0.9485
Epoch 2/10
132/132 [==============================] - 148s 1s/step - loss: 0.3157 - acc: 0.8814 - val_loss: 0.1690 - val_acc: 0.9433
Epoch 3/10
132/132 [==============================] - 147s 1s/step - loss: 0.2676 - acc: 0.8943 - val_loss: 0.1725 - val_acc: 0.9124
Epoch 4/10
132/132 [==============================] - ETA: 0s - loss: 0.2321 - acc: 0.9171

132/132 [==============================] - 155s 1s/step - loss: 0.2321 - acc: 0.9171 - val_loss: 0.1320 - val_acc: 0.9433
Epoch 5/10
132/132 [==============================] - ETA: 0s - loss: 0.2411 - acc: 0.9072

132/132 [==============================] - 153s 1s/step - loss: 0.2411 - acc: 0.9072 - val_loss: 0.0628 - val_acc: 0.9691
Epoch 6/10
132/132 [==============================] - 145s 1s/step - loss: 0.2383 - acc: 0.9019 - val_loss: 0.0748 - val_acc: 0.9742
Epoch 7/10
132/132 [==============================] - 145s 1s/step - loss: 0.1973 - acc: 0.9285 - val_loss: 0.0936 - val_acc: 0.9588
Epoch 8/10
132/132 [==============================] - 145s 1s/step - loss: 0.1675 - acc: 0.9392 - val_loss: 0.0706 - val_acc: 0.9897
Epoch 9/10
132/132 [==============================] - ETA: 0s - loss: 0.1872 - acc: 0.9338

132/132 [==============================] - 149s 1s/step - loss: 0.1872 - acc: 0.9338 - val_loss: 0.0339 - val_acc: 0.9948
Epoch 10/10
132/132 [==============================] - 146s 1s/step - loss: 0.1810 - acc: 0.9407 - val_loss: 0.0750 - val_acc: 0.9691


In [ ]:
model.save('my_model.h5')

In [ ]:
import pickle
  
# Save the trained model as a pickle string.
saved_model = pickle.dump(model, open('model.pkl', 'wb'))
  
# Load the pickled model
knn_from_pickle = pickle.loads(saved_model) 

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./model-010.h5")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('/home/user_name/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()